# Dataset Preperation

In [21]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA (GPU support) is available in PyTorch!")
    # Get the number of CUDA devices
    num_gpus = torch.cuda.device_count()
    print(f"Number of CUDA devices available: {num_gpus}")
    # Loop through the available devices and print their names
    for i in range(num_gpus):
        print(f"CUDA Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA (GPU support) is not available in PyTorch. Running on CPU.")


CUDA (GPU support) is available in PyTorch!
Number of CUDA devices available: 1
CUDA Device 0: NVIDIA GeForce RTX 3060 Laptop GPU


In [22]:
import json
import os
import re

import nltk
import pandas as pd
import sqlalchemy
from datasets import Dataset
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sqlalchemy import MetaData, create_engine
from sqlalchemy.exc import CompileError, NoReferencedColumnError
from sqlalchemy.schema import CreateTable

nltk.download("wordnet")
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saikr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saikr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saikr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
DATASET_PATH = 'spider/database'
database_paths = [(db, os.path.join(DATASET_PATH, f'{db}/{db}.sqlite')) for db in os.listdir(DATASET_PATH)]
database_schema_paths = { db[0]: os.path.join(DATASET_PATH, f'{db[0]}/schema.sql') for db in database_paths }
database_con_strings = [(db[0], f'sqlite:///{db[1]}') for db in database_paths]
print("Found Databases: ", database_con_strings)


def crawl_database(con_string, db_id):
    engine = create_engine(con_string)

    # Reflect the existing database into a new MetaData instance
    metadata = MetaData()
    metadata.reflect(bind=engine)

    # Generate 'CREATE TABLE' statements for all tables
    create_table_statements = []
    try:
        for table in metadata.sorted_tables:
            create_statement = str(CreateTable(table).compile(engine)).strip()
            create_table_statements.append(create_statement)
    except (NoReferencedColumnError, CompileError)  as e:
        with open(database_schema_paths[db_id], 'r') as f:
            create_table_statements = filter(lambda x: len(x) > 0, map(lambda x: x.strip(), f.readlines()))
    except FileNotFoundError as e:
        print(f"Schema not found for {db_id}")
        return ''
    return '\n'.join(create_table_statements)

Found Databases:  [('academic', 'sqlite:///spider/database\\academic/academic.sqlite'), ('activity_1', 'sqlite:///spider/database\\activity_1/activity_1.sqlite'), ('aircraft', 'sqlite:///spider/database\\aircraft/aircraft.sqlite'), ('allergy_1', 'sqlite:///spider/database\\allergy_1/allergy_1.sqlite'), ('apartment_rentals', 'sqlite:///spider/database\\apartment_rentals/apartment_rentals.sqlite'), ('architecture', 'sqlite:///spider/database\\architecture/architecture.sqlite'), ('assets_maintenance', 'sqlite:///spider/database\\assets_maintenance/assets_maintenance.sqlite'), ('baseball_1', 'sqlite:///spider/database\\baseball_1/baseball_1.sqlite'), ('battle_death', 'sqlite:///spider/database\\battle_death/battle_death.sqlite'), ('behavior_monitoring', 'sqlite:///spider/database\\behavior_monitoring/behavior_monitoring.sqlite'), ('bike_1', 'sqlite:///spider/database\\bike_1/bike_1.sqlite'), ('body_builder', 'sqlite:///spider/database\\body_builder/body_builder.sqlite'), ('book_2', 'sqlite

In [24]:
database_schemas = {db[0]: crawl_database(db[1], db[0]) for db in database_con_strings}
with open('database_schemas.json', 'w') as f:
    json.dump(database_schemas, f)

C:\Users\saikr\AppData\Local\Temp\ipykernel_8132\3296842037.py:13: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['11']; using no arguments.
  metadata.reflect(bind=engine)
C:\Users\saikr\AppData\Local\Temp\ipykernel_8132\3296842037.py:13: SAWarning: WARNING: SQL-parsed foreign key constraint '('Cust_ID', 'customer', 'Cust_ID')' could not be located in PRAGMA foreign_keys for table loan
  metadata.reflect(bind=engine)
C:\Users\saikr\AppData\Local\Temp\ipykernel_8132\3296842037.py:13: SAWarning: WARNING: SQL-parsed foreign key constraint '('store_id', 'store', 'store_id')' could not be located in PRAGMA foreign_keys for table staff
  metadata.reflect(bind=engine)
C:\Users\saikr\AppData\Local\Temp\ipykernel_8132\3296842037.py:13: SAWarning: WARNING: SQL-parsed foreign key constraint '('Event_ID', 'Events', 'Event_ID')' could not be located in PRAGMA foreign_keys for table Assets_in_Events
  metadata.reflect(bind=engine)
C:\Users\

In [25]:
databases = {}
type_map = []

def extract_tables_columns(tables):
    def extract_foriegn_keys(foriegn_keys):
        result = []
        for key in foriegn_keys:
            try:
                result.append((key.column.table.name, key.column.name))
            except Exception as e:
                pattern = r"'(.*?)'"
                matches = re.findall(pattern, str(key))
                if not matches:
                    matches = str(key).split('.')
                if len(matches) == 1:
                    result.append(matches[0].split('.'))
                elif len(matches) >= 2:
                    result.append((matches[0], matches[1]))
                else:
                    print(f"Error extracting foreign key: {e}")
        return result

# Finding all matches of the pattern in the input string
    result = {}
    for name, table in tables.items():
        for col in table.columns:
            col_type = str(col.type).split('(')[0]
            try:
                col_type_index = type_map.index(col_type)
            except ValueError:
                type_map.append(col_type)
                col_type_index = len(type_map) - 1
            if name not in result:
                result[name] = []
            result[name].append((col.name, col_type_index, int(col.nullable), int(col.primary_key), col.default, extract_foriegn_keys(col.foreign_keys)))
    return result


for name, db in database_con_strings:
    engine = sqlalchemy.create_engine(db)
    inspector = sqlalchemy.inspect(engine)
    metadata = sqlalchemy.MetaData()
    metadata.reflect(engine)
    try:
        tables = extract_tables_columns(metadata.tables)
    except Exception as e:
        print(f"Error extracting tables from {db}: {e}")
    databases[name] = tables

with open('databases.json', 'w') as f:
    json.dump(databases, f)

C:\Users\saikr\AppData\Local\Temp\ipykernel_8132\950617716.py:43: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['11']; using no arguments.
  metadata.reflect(engine)
C:\Users\saikr\AppData\Local\Temp\ipykernel_8132\950617716.py:43: SAWarning: WARNING: SQL-parsed foreign key constraint '('Cust_ID', 'customer', 'Cust_ID')' could not be located in PRAGMA foreign_keys for table loan
  metadata.reflect(engine)
C:\Users\saikr\AppData\Local\Temp\ipykernel_8132\950617716.py:43: SAWarning: WARNING: SQL-parsed foreign key constraint '('store_id', 'store', 'store_id')' could not be located in PRAGMA foreign_keys for table staff
  metadata.reflect(engine)
C:\Users\saikr\AppData\Local\Temp\ipykernel_8132\950617716.py:43: SAWarning: WARNING: SQL-parsed foreign key constraint '('Event_ID', 'Events', 'Event_ID')' could not be located in PRAGMA foreign_keys for table Assets_in_Events
  metadata.reflect(engine)
C:\Users\saikr\AppData\Local\Temp

In [26]:
with open('databases.json') as f:
    databases = json.loads(f.read())
    
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [t for t in tokens if t not in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]

    # Remove punctuation
    tokens = [t for t in tokens if t.isalpha()]

    # Join the tokens back into a string
    text = ' '.join(tokens)

    return text

def preprocess_dataset_entry(entry):
    """
    This function preprocesses a single dataset entry.
    It assumes `database_schemas` is a dictionary with DB schemas accessible by `db_id`.
    """
    processed_text = preprocess_text(entry['question'])
    processed_entry = {
        'input': f"translate to SQL: {processed_text} \n Schema: {json.dumps(databases.get(entry['db_id'], ''))}",
        'target': entry['query'],
    }
    return processed_entry

# Load your dataset
with open('spider/train_spider.json') as f:
    data = json.load(f)

processed_results = list(map(preprocess_dataset_entry, data))

# Convert the list of dictionaries to a pandas DataFrame
dataset_df = pd.DataFrame(processed_results)

  0%|          | 0/1970 [08:39<?, ?it/s]


In [27]:
dataset = Dataset.from_pandas(dataset_df)
dataset.save_to_disk('spider_dataset')

In [28]:
max_input_len = max(map(lambda x: len(x['input'].split()), dataset))
max_target_len = max(map(lambda x: len(x['target'].split()), dataset))
print("Max Input Length: ", max_input_len)
print("Max Target Length: ", max_target_len)
print("Dataset Size: ", len(dataset))

Max Input Length:  2181
Max Target Length:  87
Dataset Size:  7000


# Training the model

In [29]:
CKPT = 't5-small'
from transformers import AutoTokenizer, T5ForConditionalGeneration
from datasets import load_from_disk
tokenizer = AutoTokenizer.from_pretrained(CKPT)
model = T5ForConditionalGeneration.from_pretrained(CKPT)

dataset = load_from_disk('spider_dataset')

In [30]:
ds = dataset.train_test_split(test_size=0.1)
train_data = ds['train']
val_data = ds['test']

In [31]:
# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input'], pad_to_max_length=True, max_length=2048)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target'], pad_to_max_length=True, max_length=128)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [32]:
train_data = train_data.map(convert_to_features, batched=True, remove_columns=train_data.column_names)
test_data = val_data.map(convert_to_features, batched=True, remove_columns=val_data.column_names)

columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']

train_data.set_format(type='torch', columns=columns)
test_data.set_format(type='torch', columns=columns)

Map:   0%|          | 0/6300 [00:00<?, ? examples/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\saikr\anaconda3\envs\torch\Lib\site-packages\transformers\tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [33]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

# set training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="t5-small-finetuned-spider",
    auto_find_batch_size=True,
    gradient_accumulation_steps=4,
    eval_accumulation_steps=1,
    warmup_steps=2,
    max_steps=60,
    learning_rate=2e-4,
    evaluation_strategy = 'steps',
    eval_steps = 10,
    seed =  42,
    fp16=True,
    predict_with_generate=True,
    do_train=True,
    do_eval=True,
    logging_steps=500,
    save_strategy="steps",
    #save_steps=1000,
    #eval_steps=1000,
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True,
    #fp16=True,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    # compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data,
)

c:\Users\saikr\anaconda3\envs\torch\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
trainer.train()

In [1]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize the tokenizer from Hugging Face Transformers library
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained('cssupport/t5-small-awesome-text-to-sql')
model = model.to(device)
model.eval()

def generate_sql(input_prompt):
    # Tokenize the input prompt
    inputs = tokenizer(input_prompt, padding=True, truncation=True, return_tensors="pt").to(device)
    
    # Forward pass
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512)
    
    # Decode the output IDs to a string (SQL query in this case)
    generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_sql

# Test the function
#input_prompt = "tables:\n" + "CREATE TABLE Catalogs (date_of_latest_revision VARCHAR)" + "\n" +"query for: Find the dates on which more than one revisions were made."
#input_prompt = "tables:\n" + "CREATE TABLE table_22767 ( \"Year\" real, \"World\" real, \"Asia\" text, \"Africa\" text, \"Europe\" text, \"Latin America/Caribbean\" text, \"Northern America\" text, \"Oceania\" text )" + "\n" +"query for:what will the population of Asia be when Latin America/Caribbean is 783 (7.5%)?."
#input_prompt = "tables:\n" + "CREATE TABLE procedures ( subject_id text, hadm_id text, icd9_code text, short_title text, long_title text ) CREATE TABLE diagnoses ( subject_id text, hadm_id text, icd9_code text, short_title text, long_title text ) CREATE TABLE lab ( subject_id text, hadm_id text, itemid text, charttime text, flag text, value_unit text, label text, fluid text ) CREATE TABLE demographic ( subject_id text, hadm_id text, name text, marital_status text, age text, dob text, gender text, language text, religion text, admission_type text, days_stay text, insurance text, ethnicity text, expire_flag text, admission_location text, discharge_location text, diagnosis text, dod text, dob_year text, dod_year text, admittime text, dischtime text, admityear text ) CREATE TABLE prescriptions ( subject_id text, hadm_id text, icustay_id text, drug_type text, drug text, formulary_drug_cd text, route text, drug_dose text )" + "\n" +"query for:" + "what is the total number of patients who were diagnosed with icd9 code 2254?"
input_prompt = "tables:\n" + "CREATE TABLE student_course_attendance (student_id VARCHAR); CREATE TABLE students (student_id VARCHAR)" + "\n" + "query for:" + "List the id of students who never attends courses?"

generated_sql = generate_sql(input_prompt)

print(f"The generated SQL query is: {generated_sql}")
#OUTPUT: The generated SQL query is: SELECT student_id FROM students WHERE NOT student_id IN (SELECT student_id FROM student_course_attendance)


c:\Users\saikr\anaconda3\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
config.json: 100%|██████████| 2.37k/2.37k [00:00<?, ?B/s]
c:\Users\saikr\anaconda3\envs\torch\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning

The generated SQL query is: SELECT student_id FROM students WHERE NOT student_id IN (SELECT student_id FROM student_course_attendance)


In [5]:
model.save_pretrained('t5-small-finetuned-spider')
tokenizer.save_pretrained('t5-small-finetuned-spider')

('t5-small-finetuned-spider\\tokenizer_config.json',
 't5-small-finetuned-spider\\special_tokens_map.json',
 't5-small-finetuned-spider\\spiece.model',
 't5-small-finetuned-spider\\added_tokens.json')